In [1]:
from itertools import chain
from pathlib import Path
from random import sample

from import_analysis import ImportPatient
from skrt.registration import get_default_pfiles_dir, Registration, set_elastix_dir

# Define paths to patient data.
data_dir = Path("~/data/20220331_import_data_selection").expanduser()
paths = sorted(list(chain(data_dir.glob('import_high/H*'),
                          data_dir.glob('import_low/L*'))))

# Set path to Elastix installation.
elastix_dir = Path('~/sw/elastix-5.0.1').expanduser()
set_elastix_dir(elastix_dir)

# Obtain path to default Elastix parameter files.
pfiles_dir = get_default_pfiles_dir() 

In [2]:
# Load data.
patients = []
id = None
for path in paths[0: 1]:
    if id and not path.match(f"*{id}*"):
        continue
    patients.append(ImportPatient(path, load_dose_sum=False, load_masks=True))
    print(f"{patients[-1].id} - initialisation; {patients[-1]._init_time:.2f} s; ")

H01053 - initialisation; 12.57 s; 


In [3]:
# For selected patient, create registration with relapse scan as fixed image
idx = 0
p = patients[idx]

# Crop relapse scan to include structure-set ROIs plus margin.
ct_relapse = p.get_ct_relapse().clone()
ct_relapse.crop_to_roi(p.get_ss_relapse() + p.get_ss_recurrence(), buffer=100)

# Crop planning scan to size of cropped relapse scan.
ct_plan = p.get_ct_plan().clone()
ct_plan.crop_to_image(ct_relapse, alignment="spinal_canal")

# Define registration strategy.
reg = Registration(
    Path("results/relapse_fixed") / p.id,
    fixed=ct_relapse,
    moving=ct_plan,
    fixed_mask=None,
    initial_alignment = None,
    pfiles={
        #"translation": pfiles_dir / "MI_Translation.txt",
        "rigid": pfiles_dir / "MI_Rigid.txt",
        "affine": pfiles_dir / "MI_Affine.txt",
        #"bspline": pfiles_dir / "MI_BSpline30.txt"
    },
    overwrite=True,
    capture_output=True
)

In [4]:
# Perform registration.
reg.register()

In [5]:
# Show results at each step of registration.
for step in reg.steps:
    print(step)
    reg.view_result(step)

rigid


Output()

affine


Output()

In [6]:
# Push ROI contours from relapse frame to planning frame.
ss_relapse_transformed = reg.transform(p.get_ss_relapse(), transform_points=True)
ss_recurrence_transformed = reg.transform(p.get_ss_recurrence(), transform_points=True)
ss_recurrence_transformed["recurrence"].name = "Transformed_recurrence"

In [ ]:
# Show relapse scan with ROI contours outlined on scan,
# and ROI contours pushed from planning scan.
reg.moving_image.view(rois=ss_relapse_transformed + p.get_ss_plan())

In [8]:
# Show recurrence mapped to planning scan and on relapse scan.
ct_plan.view(images=ct_relapse, rois=[ss_recurrence_transformed, p.get_ss_recurrence()])

Output()